In [14]:
from helpers import *
import pickle
import pandas as pd
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
with open("dataset/RateBeer/ratings_reviews.pickle", 'rb') as file:
    ratings = pickle.load(file)

In [61]:
breweries = pd.read_csv("dataset/RateBeer/breweries.csv")
users = pd.read_csv("dataset/RateBeer/users.csv")

In [8]:
ratings['date'] = ratings['date'].apply(convert_unix_timestamp)

In [56]:
year_filter = ratings['date'].dt.year.isin([2010, 2011, 2012])
ratings_reviews_2010_2012 = ratings.loc[year_filter]

In [62]:
ratings_with_user_location = pd.merge(ratings_reviews_2010_2012, 
                                              users[['user_id', 'location']], on='user_id')

In [63]:
breweries = breweries.rename(columns={'location': 'brewery_location'})
ratings_with_user_brewery_location = pd.merge(ratings_with_user_location, breweries[['brewery_location', 'id']], left_on='brewery_id', right_on='id')

In [64]:
ratings_with_user_brewery_location

,beer_name,beer_id,brewery_name,brewery_id,style,abv,date,user_name,user_id,appearance,aroma,palate,taste,overall,rating,text,location,brewery_location,id
0,Régab,19445,Sobraga,3198,Pale Lager,4.5,2012-11-12 11:00:00,fonefan,38926,2,3,2,4,7,1.8,Bottle. [ As my country rating # 134 on ratebe...,Denmark,Gabon,3198
1,Régab,19445,Sobraga,3198,Pale Lager,4.5,2012-09-17 10:00:00,Kingink,217452,4,5,3,6,7,2.5,"As lagers go, very drinkable, especially in eq...",Scotland,Gabon,3198
2,Régab,19445,Sobraga,3198,Pale Lager,4.5,2012-09-06 10:00:00,Zita,98909,3,3,2,2,5,1.5,"light sweet aroma,yellow,clear, white foam, li...",Denmark,Gabon,3198
3,Régab,19445,Sobraga,3198,Pale Lager,4.5,2012-09-06 10:00:00,Hansen,5986,2,3,2,4,7,1.8,Yellow coloured and almost clear. Has a white ...,Denmark,Gabon,3198
4,Régab,19445,Sobraga,3198,Pale Lager,4.5,2012-09-06 10:00:00,onystrom,26684,2,4,1,4,5,1.6,Clear yellow color with small head. Some bitte...,Denmark,Gabon,3198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1436910,Holy Cow Pumpkin Ale,5274,"Holy Cow&#033; Casino, Cafe and Brewery",575,Spice/Herb/Vegetable,NaN,2010-11-22 11:00:00,Rnippoldt13,117691,4,10,5,8,17,4.4,I tasted this beer many years ago and now Holy...,NaN,"United States, Nevada",575
1436911,Cyclers 55-11 Imperial Red,191802,Cyclers Brewing Company,15561,American Strong Ale,7.9,2012-11-26 11:00:00,cmatt,226054,4,8,4,8,18,4.2,"Awesome imperial red taste! Murky, but well b...","United States, Texas","United States, Texas",15561
1436912,Grand Union One Hop: Magnum,65135,Grand Union,2894,Bitter,4.5,2010-12-31 11:00:00,bluekuma,33112,3,4,4,7,17,3.5,"Cask conditioned, gravity dispense at St. Alba...",England,England,2894
1436913,Bridgetown Albert Ale,92104,Bridgetown,9820,Bitter,3.8,2011-08-24 10:00:00,cellargenius,134182,5,8,5,8,19,4.5,"fantastic rustic traditional beer, drunk 20 me...",England,England,9820


In [65]:
ratings_with_brewery_location_country_population = ratings_with_user_brewery_location.groupby(['location', 'brewery_location'])
beer_country_location = ratings_with_brewery_location_country_population.size().reset_index(name = 'number')

In [68]:
beer_country_location

,location,brewery_location,number
0,Abkhazia,Germany,2
1,Afghanistan,Belgium,1
2,Afghanistan,Italy,1
3,Afghanistan,"United States, California",1
4,Albania,England,1
...,...,...,...
10563,Wales,"United States, Illinois",1
10564,Wales,"United States, Maryland",1
10565,Wales,"United States, Missouri",1
10566,Wales,Vietnam,1


In [67]:
ratings_reviews_groupedby_loc = ratings_with_user_location.groupby('location').size().reset_index(name = 'total_loc_number')
ratings_reviews_groupedby_loc

,location,total_loc_number
0,Abkhazia,2
1,Afghanistan,3
2,Albania,1
3,Andorra,25
4,Argentina,83
...,...,...
164,Uruguay,81
165,Venezuela,1156
166,Vietnam,13
167,Virgin Islands (U.S.),5


In [69]:
location_country = pd.merge(beer_country_location, ratings_reviews_groupedby_loc, on = 'location', how = 'inner')

In [70]:
location_country['popularity_percentage'] = 100* (location_country['number'] / location_country['total_loc_number'])

In [71]:
location_country

,location,brewery_location,number,total_loc_number,popularity_percentage
0,Abkhazia,Germany,2,2,100.000000
1,Afghanistan,Belgium,1,3,33.333333
2,Afghanistan,Italy,1,3,33.333333
3,Afghanistan,"United States, California",1,3,33.333333
4,Albania,England,1,1,100.000000
...,...,...,...,...,...
10563,Wales,"United States, Illinois",1,325,0.307692
10564,Wales,"United States, Maryland",1,325,0.307692
10565,Wales,"United States, Missouri",1,325,0.307692
10566,Wales,Vietnam,1,325,0.307692
